In [44]:
!pip install torch torchtext transformers sentencepiece tqdm datasets pandas


In [45]:
import pandas as pd
import ast
from datasets import Dataset,DatasetDict,load_dataset
from tqdm import tqdm
import datasets



In [46]:
#load the dataset
data_sample = load_dataset("QuyenAnhDE/Diseases_Symptoms")


Repo card metadata block was not found. Setting CardData to empty.


In [47]:
data_sample

DatasetDict({
    train: Dataset({
        features: ['Code', 'Name', 'Symptoms', 'Treatments'],
        num_rows: 400
    })
})

In [48]:
updated_data = [{'Name' : item['Name'],'Symptoms':item['Symptoms']} for item in data_sample['train']]

In [49]:
import pandas as pd
df = pd.DataFrame(updated_data)
df.head(5)

,Name,Symptoms
0,Panic disorder,"Palpitations, Sweating, Trembling, Shortness o..."
1,Vocal cord polyp,"Hoarseness, Vocal Changes, Vocal Fatigue"
2,Turner syndrome,"Short stature, Gonadal dysgenesis, Webbed neck..."
3,Cryptorchidism,"Absence or undescended testicle(s), empty scro..."
4,Ethylene glycol poisoning-1,"Nausea, vomiting, abdominal pain, General mala..."


In [50]:
df['Symptoms'] = df['Symptoms'].apply(lambda x: ','.join(x.split(', ')))

In [51]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split


In [52]:
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

In [53]:
device

device(type='cuda')

In [54]:
tokenizer = GPT2Tokenizer.from_pretrained("distilbert/distilgpt2")
model = GPT2LMHeadModel.from_pretrained("distilbert/distilgpt2").to(device)


Loading weights:   0%|          | 0/76 [00:00<?, ?it/s]

GPT2LMHeadModel LOAD REPORT from: distilbert/distilgpt2
Key                                        | Status     |  | 
-------------------------------------------+------------+--+-
transformer.h.{0, 1, 2, 3, 4, 5}.attn.bias | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


In [55]:
model


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-5): 6 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [56]:
BATCH_SIZE = 8

In [57]:
df.describe()

,Name,Symptoms
count,400,400
unique,392,395
top,Sciatica,"Swelling,pain,dry mouth,bad taste"
freq,3,3


In [58]:
#Dataset Preperation
class Languagedataset(Dataset):
  def __init__(self,df,tokenizer):
    self.df = df
    self.labels = df.columns
    self.data = df.to_dict(orient='records')
    self.tokenizer = tokenizer
    x = self.fittest_max_length(df)
    self.max_length = x
#Length of the dataset
  def __len__(self):
    return len(self.data)
#Getting one trainning sample
  def __getitem__(self, idx):
    x = self.data[idx][self.labels[0]]
    y = self.data[idx][self.labels[1]]
    text = f"{x} | {y}"
    tokens = self.tokenizer(  # ✅ Use __call__ method
        text,
        return_tensors='pt',  # ✅ Also fixed typo: return_tensors not return_tensor
        max_length=128,
        padding='max_length',
        truncation=True
    )
    return tokens

  def fittest_max_length(self,df):
    max_length = max(len(max(df[self.labels[0]],key = len)),
                     len(max(df[self.labels[1]],key = len))
                     )
    x = 2
    while x < max_length:x = x*2
    return x




In [59]:
data_sample = Languagedataset(df,tokenizer)


In [60]:
data_sample


In [61]:
train_size = int(0.8* len(data_sample))
val_size = len(data_sample) - train_size
train_data,val_data = random_split(data_sample,[train_size,val_size])



In [62]:
#Make the iterators
train_loader = DataLoader(train_data,batch_size=BATCH_SIZE,shuffle=True)
val_loader = DataLoader(val_data,batch_size=BATCH_SIZE)




In [63]:
num_epochs = 10

In [64]:
batch_size = BATCH_SIZE
model_name = "distilgpt2"
gpu = 0

In [65]:
#Loss function and Optimizer
criterion = nn.CrossEntropyLoss(ignore_index=tokenizer.pad_token_type_id)
optimizer = optim.Adam(model.parameters(),lr=5e-4)
tokenizer.pad_token = tokenizer.eos_token


In [66]:
results = pd.DataFrame(columns=['epoch','transfomer','batch_size','gpu','trainning_loss','val_loss','epoch_duration_sec'])

In [71]:
import time

for epoch in range(num_epochs):
    start_time = time.time()

    # ================= TRAIN =================
    model.train()
    epoch_training_loss = 0

    train_iterator = tqdm(
        train_loader,
        desc=f"Training epoch {epoch+1}/{num_epochs}"
    )

    for batch in train_iterator:
        optimizer.zero_grad()

        inputs = batch['input_ids'].to(device)
        if inputs.dim() == 3:
            inputs = inputs.squeeze(1)

        outputs = model(input_ids=inputs, labels=inputs)
        loss = outputs.loss

        loss.backward()
        optimizer.step()

        epoch_training_loss += loss.item()
        train_iterator.set_postfix(loss=loss.item())

    avg_epoch_training_loss = epoch_training_loss / len(train_loader)

    # ================= VALIDATION =================
    model.eval()
    epoch_val_loss = 0

    with torch.no_grad():
        valid_iterator = tqdm(val_loader, desc=f"Validating epoch {epoch+1}/{num_epochs}")

        for batch in valid_iterator:
            inputs = batch['input_ids'].to(device)
            if inputs.dim() == 3:
                inputs = inputs.squeeze(1)

            outputs = model(input_ids=inputs, labels=inputs)
            loss = outputs.loss

            epoch_val_loss += loss.item()
            valid_iterator.set_postfix(loss=loss.item())

    avg_epoch_val_loss = epoch_val_loss / len(val_loader)

    epoch_duration = time.time() - start_time

    results.loc[len(results)] = {
        'epoch': epoch+1,
        'transformer': model_name,
        'batch_size': batch_size,
        'gpu': gpu,
        'trainning_loss': avg_epoch_training_loss,
        'val_loss': avg_epoch_val_loss,
        'epoch_duration_sec': epoch_duration
    }

    print(
        f"Epoch {epoch+1} | "
        f"Train Loss: {avg_epoch_training_loss:.4f} | "
        f"Val Loss: {avg_epoch_val_loss:.4f} | "
        f"Time: {epoch_duration:.2f}s"
    )


Validating epoch 1/10: 100%|██████████| 10/10 [00:00<00:00, 15.81it/s, loss=0.819]


Epoch 1 | Train Loss: 0.0763 | Val Loss: 1.0015 | Time: 9.26s


Validating epoch 2/10: 100%|██████████| 10/10 [00:00<00:00, 16.12it/s, loss=0.868]


Epoch 2 | Train Loss: 0.0657 | Val Loss: 1.0577 | Time: 9.50s


Validating epoch 3/10: 100%|██████████| 10/10 [00:00<00:00, 16.41it/s, loss=0.863]


...

Epoch 8 | Train Loss: 0.0455 | Val Loss: 1.1493 | Time: 8.63s


Validating epoch 9/10: 100%|██████████| 10/10 [00:00<00:00, 16.92it/s, loss=0.986]


Epoch 9 | Train Loss: 0.0461 | Val Loss: 1.1588 | Time: 8.74s


Validating epoch 10/10: 100%|██████████| 10/10 [00:00<00:00, 16.76it/s, loss=0.959]

Epoch 10 | Train Loss: 0.0395 | Val Loss: 1.1693 | Time: 8.85s


In [87]:
input_str = "Depression"

In [88]:
input_ids = tokenizer.encode(input_str, return_tensors='pt').to(device)

In [89]:
input_ids

tensor([[12156,  2234]], device='cuda:0')

In [90]:
output = model.generate(input_ids, max_length=20, num_return_sequences=1, do_sample = True,top_k = 8,top_p = 0.95,temperature = 0.5)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [91]:
output

tensor([[12156,  2234, 31162,   930, 25723, 26728,    11,   805,   291,  8640,
            11, 10378,  3314,  8640,    11,   343,   799,  1799,    11, 26069]],
       device='cuda:0')

In [92]:
decoded_output = tokenizer.decode(output[0], skip_special_tokens=True)

In [93]:
decoded_output

'Depression Disorder | Mood swings,manic episodes,depressive episodes,irritability,diff'

In [94]:
torch.save(model,"SmallDiseasesLM.pt")

In [96]:
torch.save(model,'drive/My Drive/SmallDiseasesLM.pt')